# ML/XAI Tool Chest

## Import Packages

In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)
import numpy as np
import pprint
import seaborn as sns
from datetime import datetime, timezone, timedelta
import time
import pytz
import re
import matplotlib.pyplot as plt
from random import randint
#import pickle
import sklearn.model_selection as model
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# classifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import xgboost as xgb
# hyper-parameter tuning
from scipy.stats import randint as sp_randint
import sys
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler, NearMiss, TomekLinks
from imblearn.combine import SMOTETomek

In [2]:
import warnings
warnings.filterwarnings('ignore')

# 1) Data Loading & Splitting

In [3]:
# load data and split into train and test sets
def split_data(df, target_var, total_record_cnt=-1, train_size=0.7, test_size=0.3, valid_size=0, rand_state=42):

    # error catch - validate input parameters    
    if (train_size + test_size + valid_size) != 1:
        raise ValueError("Selected sample sizes don't add up to 1! Please correct.")

    # random sample (if selected)
    if total_record_cnt>0:
        df = df.sample(n=size, replace=False, random_state=rand_state)
    
    # X and y splitting
    y = df[target_var]
    X = df.drop(target_var, axis=1)

    if valid_size>0:
        # train-test-valid splitting
        X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=train_size)
        X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=(test_size/(test_size+valid_size)), random_state=42)

        # User message
        print('\n\033[1mTrain-Test-Valid Splitting Results:\033[0m\n')
        print(f'Training: \t{100*train_size}%, \t{X_train.shape[0]} samples.')
        print(f'Test: \t\t{100*test_size}%,  {X_test.shape[0]} samples.')
        print(f'Validation: \t{100*valid_size}%,  \t{X_valid.shape[0]} samples.\n')
        
        return X_train, X_test, X_valid, y_train, y_test, y_valid

    else:
        # train-test splitting
        X_train, X_test, y_train, y_test = model.train_test_split(X, y, train_size=train_size, random_state=42)

        # User message
        print('\n\033[1mTrain-Test Splitting Results:\033[0m\n')
        print(f'Training: \t{100*train_size}%, \t{X_train.shape[0]} samples.')
        print(f'Test: \t\t{100*test_size}%,  {X_test.shape[0]} samples.\n')
        
        return X_train, X_test, y_train, y_test

# 2) Data Preprocessing

## a) Class Balancing

In [4]:
# apply over/under-sampling methods for class balancing
def class_balancer(X, y, method, sampling_strategy=1, version=1, n_neighbors=3):

    # error catch - validate input parameters
    if sampling_strategy not in ['SMOTE', 'ClusterCentroids', 'RandomUnder', 'NearMiss', 'Tomek', 'SMOTETomek']:
        raise ValueError("Please select a sampling strategy! Choose between 'SMOTE', 'ClusterCentroids', 'RandomUnder', 'NearMiss', 'Tomek', or 'SMOTETomek'.")
    
    ### OVER-SAMPLING ###
    # SMOTE
    if method=="SMOTE":    
        sampler = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
    
    ### UNDER-SAMPLING ###
    # ClusterCentroid
    elif method=="ClusterCentroids":
        sampler = ClusterCentroids(sampling_strategy=sampling_strategy, random_state=42)
    # RandomUnderSampler
    elif method=="RandomUnder":
        sampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
    # NearMiss
    elif method=="NearMiss":
        sampler = NearMiss(sampling_strategy=sampling_strategy, version=version, n_neighbors=n_neighbors)
    # TomekLinks
    elif method=="Tomek":
        sampler = TomekLinks(sampling_strategy=sampling_strategy)

    ### HYBRID ###
    # SMOTE-Tomek
    elif method=="SMOTETomek":
        sampler = SMOTETomek(sampling_strategy=sampling_strategy, random_state=42)
    
    # apply sampling method
    X, y = sampler.fit_resample(X, y)
    
    return X, y

## b) Feature Scaling (Standardization & Normalization)

In [5]:
# apply normalization or standardization scaling and return scaled data as well as scaler
def scaler_fit_and_transform(df, cols, scaler):

    # error catch - validate input parameters
    if scaler not in ['MinMax', 'Standard']:
        raise ValueError("Please select a valid scaler! Choose between 'MinMax' or 'Standard'.")
    
    # create copy of input data
    out = df.copy()

    # initialize scaler
    if scaler == 'MinMax':
        scaler = MinMaxScaler()
    elif scaler == 'Standard':
        scaler = StandardScaler()

    # fit scaler and transform data
    out[cols] = scaler.fit_transform(out[cols])
    #out = pd.DataFrame(out, columns=df.columns)

    # User message
    print('\n\033[1mFeature Scaling Results - Training Data:\033[0m\n')
    print(f'The following numerical features were scaled using the "{scaler}":\n')
    print('"'+', '.join(cols)+'"\n')
    display(out.describe())
        
    return out, scaler

In [6]:
# use already fitted scaler to e.g. scale test set or new observations
def scaler_transform(df, cols, scaler_fitted):

    # create copy of input data
    out = df.copy()
    
    # transform data using fitted scaler
    out[cols] = scaler_fitted.transform(out[cols])
    #out = pd.DataFrame(out, columns=df.columns)

    # User message
    print('\n\033[1mFeature Scaling Results - Test/Validation Data:\033[0m\n')
    print(f'The following numerical features were scaled using the pre-fitted "{scaler}":\n')
    print('"'+', '.join(cols)+'"\n')
    display(out.describe())
        
    return out

## c) Train/Test-Data Selection

In [7]:
# select the respective datasets based on selected scaling and sampling method as well as sampling size 
def select_X_y(X_train, y_train, X_test, y_test, sampler=None, scaler=None, total_record_cnt=-1,
               sampling_strategy=1, version=0, n_neighbors=0):

    # apply class balancer function
    if sampler:
        X_train, y_train = class_balancer(X_train, y_train, sampler, sampling_strategy, version, n_neighbors)

    # apply scaler function
    if scaler:
        X_train, scaler_fitted = scaler_fit_transform(X_train, cols_to_scale, scaler)
        X_test = scaler_transform(X_test, cols_to_scale, scaler_fitted)
            
    # reduce train-sample size
    if total_record_cnt>0:
        X_train = X_train.sample(n=size, replace=False, random_state=42)
        y_train = y_train[y_train.index]

    return X_train, y_train, X_test, y_test

## d) One-Hot-Encoding

In [36]:
# apply one-hot encoding on categorical features
def one_hot_encoding(df, cols, drop_first=True, dtype=bool, info=True):

    # create copy of input dataframe
    out = df.copy()
    
    for col in cols:
        out = pd.get_dummies(data=out, columns=[col], drop_first=drop_first, prefix=col, dtype=dtype)

    if info:
        # User message
        print('\n\033[1mOne-Hot Encoding Results:\033[0m\n')
        print(f'The following categorical features were encoded:\n')
        print('"'+', '.join(cols)+'"\n')
        display(out.head(3))
    
    return out

## e) Outlier Removal

In [29]:
def outlier_removal(X, y, cols, detection_method='iqr', thres=1.5, handling_method='cap'):

    # error catch - validate input parameters
    if detection_method not in ['iqr', 'std', 'zscore']:
        raise ValueError("Please select a valid detection method! Choose betwen 'iqr', 'std', or 'zscore'.")

    if handling_method not in ['drop', 'cap']:
        raise ValueError("Please select a valid handling method! Choose between 'drop' or 'cap'.")
    
    # create copy of input dataframe
    X_temp = X.copy()
    y_temp = y.copy()

    # List to store outlier indices across all columns
    ind_all_outliers = []

    # looping through passed on columns
    for col in cols:

        # outlier detection
        ## IQR method
        if detection_method == 'iqr':
            q1 = X_temp[col].quantile(0.25)
            q3 = X_temp[col].quantile(0.75)
            iqr = q3 - q1
            upper = q3 + (thres * iqr)
            lower = q1 - (thres * iqr)
            ind_col_outlier = X_temp[(X_temp[col] > upper) | (X_temp[col] < lower)].index
      
        ## standard-deviation method
        elif detection_method == 'std':
            mean = np.mean(X_temp[col])
            std = np.std(X_temp[col])
            upper = mean + (thres * std)
            lower = mean - (thres * std)
            ind_col_outlier = X_temp[(X_temp[col] > upper) | (X_temp[col] < lower)].index
            
        ## z-score method
        elif detection_method == 'zscore':
            mean = np.mean(X_temp[col])
            std = np.std(X_temp[col])
            ind_col_outlier = X_temp[abs((X_temp[col] - mean) / std) > thres].index

        
        # outlier handling
        ## dropping outliers
        if handling_method == 'drop':
            X_temp = X_temp.drop(ind_col_outlier)
            y_temp = y_temp.drop(ind_col_outlier)
        ## capping outliers to max value
        elif handling_method == 'cap':
            X_temp.loc[ind_col_outlier, col] = upper

        # Add the indices of outliers for the current column to the overall list
        ind_all_outliers.extend(ind_col_outlier)

    # User message
    print('\n\033[1mOutlier Removal Results:\033[0m\n')
    print(f'Detected {len(ind_all_outliers)} outliers using the "{detection_method}" approach.')
    print(f'Outliers have been "{handling_method}ped".\n')
    print(f'The dataset now has {X_temp.shape[0]} samples.\n')

    return X_temp, y_temp

## f) Missing Data Handling

In [28]:
def check_missing_data(df):

    # Count of missing values per column
    missing_count = df.isnull().sum()

    # Percentage of missing values per column
    missing_percentage = (df.isnull().sum() / len(df)) * 100

    # Combine count and percentage into a single DataFrame for a better display
    print('\n\033[1mMissing Data Analysis Results:\033[0m\n')
    missing_data = pd.DataFrame({'Missing Count': missing_count, 'Percentage (%)': missing_percentage})

    return missing_data

# 3) Data Modeling

## a) Model Fitting

#### i) "White Box" Algorithms

In [10]:
# fit classifier/regressor
def fit_model(X, y, algo):
    
    # create copy of input classifier
    model = algo
    
    # fit the model
    model = model.fit(X, y)
    
    return model

#### ii) Neural Network ("Black Box")

In [11]:
# learning evolution plot
def plot_learning_evolution(clf):
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.plot(clf.history['loss'], label='Loss')
    plt.plot(clf.history['val_loss'], label='val_Loss')
    plt.title('Loss evolution during trainig')
    plt.legend()

    plt.subplot(2, 2, 2)
    plt.plot(clf.history['AUC'], label='AUC')
    plt.plot(clf.history['val_AUC'], label='val_AUC')
    plt.title('AUC score evolution during trainig')
    plt.legend()

# fit neural network
def fit_nn(X_train, y_train, X_test, y_test, num_cols, num_labels, hidden_units, dropout_rates, 
           learn_rate, epochs=20, batch_size=32):
    
    inp = tf.keras.layers.Input(shape=(num_cols))
    x = BatchNormalization()(inp)
    x = Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = Dense(hidden_units[i], activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(dropout_rates[i + 1])(x)
    x = Dense(num_labels, activation='sigmoid')(x)
  
    clf = Model(inputs=inp, outputs=x)
    clf.compile(optimizer=Adam(learn_rate), loss='binary_crossentropy', metrics=[AUC(name='AUC')])
    
    clf = clf.fit(X_train, y_train,
                  validation_data=(X_test, y_test),
                  epochs=epochs,
                  batch_size=batch_size)
    
    #plot_learning_evolution(clf)
    
    return clf

## b) Model Deployment / Predicting

In [12]:
# make predictions using fitted classifier
def predict_clf(X, clf, thres=0.5):
    
    # make predcitions
    y_prob = clf.predict_proba(X)[:,1]
    y_pred = [1 if x >= thres else 0 for x in y_prob]
    
    return y_pred, y_prob

## c) Parameter Optimization

### i) Optimal Classifier Threshold

In [13]:
# return summary table that shows performance metrics across different classifier threshold levels
def grid_search_thres(clf, X_test, y_test, step=0.01, show=False):
    
    # initialize output table
    thres_grid = pd.DataFrame(index=['Accuracy', 'Balanced Acc', 'Precision', 'Recall', 'F1','AUC'])

    # stepwise loop through threshold levels
    for i in np.arange(0, 1.0, step):
        # assign target labels based on current threshold
        y_pred, y_prob = predict(X_test, clf, thres=i)
        
        # add metrics to output table
        thres_grid[i] = [round(metrics.accuracy_score(y_test, y_pred),4),
                         round(metrics.balanced_accuracy_score(y_test, y_pred),4),
                         round(metrics.precision_score(y_test, y_pred),4),
                         round(metrics.recall_score(y_test, y_pred),4),
                         round(metrics.f1_score(y_test, y_pred, average='weighted'),4),
                         round(metrics.roc_auc_score(y_test, y_prob),4)]
        
    if show: display(thres_grid)
            
    return thres_grid

In [14]:
# compute the optimal threshold level using Youden's J statistic or Precision-Recall Curve approach
def get_opt_thres(clf, X_test, y_test):
    
    y_prob = clf.predict_proba(X_test)[:,1]
    
    # Youden’s J statistic
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
    score = tpr - fpr
    
    # Precision-Recall Curve
    """
    precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_prob)
    score = (2 * precision * recall) / (precision + recall)
    """
    
    i = np.argmax(score)
    opt_thres = thresholds[i]
    print('Optimal Threshold: %.4f' % (opt_thres))
    
    return opt_thres

In [15]:
# plot all evaluation metrics with moving threshold level
def plot_thres(clf, X_test, y_test, step=0.01, metric=None):
            
    # threshold grid search table
    thres_grid = grid_search_thres(clf, X_test, y_test, step)

    # get optimal threshold
    opt_thres = get_opt_thres(clf, X_test, y_test)
    
    # plot curve for selected metric
    if metric != None:
        temp = pd.DataFrame(thres_grid.loc[metric]).reset_index()
        temp.rename(columns = {'index':'Threshold'}, inplace = True)
        plt.ylim(0, 1)
        sns.lineplot(data=temp, x='Threshold', y=metric).set(title='Threshold Tuning Curve for '+metric)
        plt.axvline(opt_thres, color='r')
        plt.text(opt_thres+0.03, 0.9,'t=%.2f' % opt_thres, color='r')
        
    # plot curves for all metrics
    else:
        i = 1
        m_list = list(thres_grid.index.values)
        del m_list[-1]
        for m in m_list:
            temp = pd.DataFrame(thres_grid.loc[m]).reset_index()
            temp.rename(columns = {'index':'Threshold'}, inplace = True)
            plt.figure(figsize=(15, 5))
            plt.subplot(3, 2, i)
            plt.ylim(0, 1)
            sns.lineplot(data=temp, x='Threshold', y=m).set(title='Threshold Tuning Curve for '+m)
            plt.axvline(opt_thres, color='r')
            plt.text(opt_thres+0.03, 0.9,'t=%.2f' % opt_thres, color='r')
            i += 1

### ii) Hyper-Parameters: Random Search

In [16]:
# hyper-parameter tuning using random search
def random_search(X, y, clf, params, n_iter=5, cv=3, k_fold=3, scoring='balanced_accuracy'):
    
    # shuffle splitting
    cv = model.ShuffleSplit(n_splits=k_fold, test_size=0.3, random_state=42)
    
    # define random search parameters
    randomCV = model.RandomizedSearchCV(clf, param_distributions=params, n_iter=n_iter, cv=cv, scoring=scoring, random_state=42)
    
    # fit models
    randomCV.fit(X, y)
    
    return randomCV.best_estimator_, randomCV.best_params_, randomCV.cv_results_

### iii) Hyper-Parameters: Bayesian Optimization

In [17]:
#hyper-parameter tuning using bayesian optimization
def bayesian_opt():
    
    return 1

### iv) Optimal Dataset Parameters

In [18]:
# find optimal sampling and scaling method for specific classifier using specified eval metric
def get_optimal_data(df, clf, metric='AUC'):
    
    # find index for optimal score for selected clf
    idx = df.loc[df['Clf']==clf][[metric]].idxmax()
    
    # extract optimal parameters
    scaler = str(df.loc[idx, 'Scaling'][0])
    sampler = str(df.loc[idx, 'Sampling'][0])
    
    if scaler=='non':
        scaler = None
    if sampler=='non':
        sampler = None
    
    return scaler, sampler 

## d) Cross-Validation

In [19]:
# cross-validate model
def cv_model(X, y, clf, clf_name, k_fold=3):
    
    # shuffle split
    cv = model.ShuffleSplit(n_splits=k_fold, test_size=0.3, random_state=42)

    # initialize scoring metrics and summary tables
    scoring = ['accuracy','balanced_accuracy','precision','recall','f1','roc_auc']
    model_eval = pd.DataFrame(index=['Accuracy', 'Bal_Acc', 'Precision', 'Recall', 'F1', 'AUC'])
    model_eval_detail = pd.DataFrame(index=['Fold', 'Fit_Time', 'Score_Time', 'Accuracy', 'Bal_Acc', 'Precision', 'Recall', 'F1', 'AUC'])

    cv_scores = model.cross_validate(clf, X, y, cv=cv, scoring=scoring, return_estimator=True)

    # CV aggegrated scores
    model_eval[clf_name] = [round(cv_scores['test_accuracy'].mean(),4),
                            round(cv_scores['test_balanced_accuracy'].mean(),4),
                            round(cv_scores['test_precision'].mean(),4),
                            round(cv_scores['test_recall'].mean(),4),
                            round(cv_scores['test_f1'].mean(),4),
                            round(cv_scores['test_roc_auc'].mean(),4)]
    
    # CV scores for each fold
    model_eval_detail[clf_name] = [list(range(1,len(cv_scores['test_accuracy'])+1)),
                                   np.round(cv_scores['fit_time'],2),
                                   np.round(cv_scores['score_time'],2),
                                   np.round(cv_scores['test_accuracy'],4),
                                   np.round(cv_scores['test_balanced_accuracy'],4),
                                   np.round(cv_scores['test_precision'],4),
                                   np.round(cv_scores['test_recall'],4),
                                   np.round(cv_scores['test_f1'],4),
                                   np.round(cv_scores['test_roc_auc'],4)]
    
    #display(model_eval)
    #display(model_eval_detail)

    return model_eval, model_eval_detail

## e) Model Evaluation

### (i) Classification

In [20]:
# evaluate (final) model
def eval_model(X_test, y_test, X_train, y_train, clf_fitted, clf_name, thres=0.5, show=True):
    
    # initialize scoring metrics and summary table
    scoring = ['accuracy','balanced_accuracy','precision','recall','f1','roc_auc']
    model_eval = pd.DataFrame(index=['RunTime', 'Accuracy', 'Bal_Acc', 'Precision', 'Recall', 'F1', 'AUC'])
    
    # get preds and probs
    start_time = time.time()
    y_pred_train, y_prob_train = predict(X_train, clf_fitted, thres)
    train_time = time.time()
    y_pred_test, y_prob_test = predict(X_test, clf_fitted, thres)
    test_time = time.time()
    
    # model scores
    model_eval[clf_name+'_train'] = [str(timedelta(seconds=train_time-start_time)),
                                     round(metrics.accuracy_score(y_train, y_pred_train),4),
                                     round(metrics.balanced_accuracy_score(y_train, y_pred_train),4),
                                     round(metrics.precision_score(y_train, y_pred_train),4),
                                     round(metrics.recall_score(y_train, y_pred_train),4),
                                     round(metrics.f1_score(y_train, y_pred_train, average='weighted'),4),
                                     round(metrics.roc_auc_score(y_train, y_prob_train),4)]
    
    model_eval[clf_name+'_test'] = [str(timedelta(seconds=test_time-train_time)),
                                    round(metrics.accuracy_score(y_test, y_pred_test),4),
                                    round(metrics.balanced_accuracy_score(y_test, y_pred_test),4),
                                    round(metrics.precision_score(y_test, y_pred_test),4),
                                    round(metrics.recall_score(y_test, y_pred_test),4),
                                    round(metrics.f1_score(y_test, y_pred_test, average='weighted'),4),
                                    round(metrics.roc_auc_score(y_test, y_prob_test),4)]
    
    
    display(model_eval.transpose())
    
    # confusion matrix and ROC curve
    cm = metrics.ConfusionMatrixDisplay.from_estimator(clf_fitted, X_test, y_test, cmap='cividis', display_labels=['Fully-Paid','Default'])
    roc = metrics.RocCurveDisplay.from_estimator(clf_fitted, X_test, y_test)
    
    return model_eval.transpose(), cm, roc

### (ii) Regression

In [ ]:
def evaluate_reg_model(model_name, y_train, y_test, y_pred_train, y_pred_test, n_decimals=10):

    # Evaluate the model
    ## R2
    train_r2 = round(r2_score(y_train, y_pred_train), n_decimals)
    test_r2 = round(r2_score(y_test, y_pred_test), n_decimals)
    ## Adj. R2
    train_adj_r2 = round(1 - (1 - r2_score(y_train, y_pred_train)) * (X_train.shape[0] - 1) / (X_train.shape[0] - X_train.shape[1] - 1), n_decimals)
    test_adj_r2 = round(1 - (1 - r2_score(y_test, y_pred_test)) * (X_test.shape[0] - 1) / (X_test.shape[0] - X_test.shape[1] - 1), n_decimals)
    ## RSS
    train_rss = round(np.sum(np.square(y_train-y_pred_train)), n_decimals)
    test_rss = round(np.sum(np.square(y_test-y_pred_test)), n_decimals)
    ## MSE
    train_mse = round(mean_squared_error(y_train, y_pred_train), n_decimals)
    test_mse = round(mean_squared_error(y_test, y_pred_test), n_decimals)
    ## RMSE
    train_rmse = round(np.sqrt(mean_squared_error(y_train, y_pred_train)), n_decimals)
    test_rmse = round(np.sqrt(mean_squared_error(y_test, y_pred_test)), n_decimals)
    ## MAE
    train_mae = round(mean_absolute_error(y_train, y_pred_train), n_decimals)
    test_mae = round(mean_absolute_error(y_test, y_pred_test), n_decimals)
    ## MAPE
    train_mape = round(np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100, n_decimals)
    test_mape = round(np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100, n_decimals)

    # Store the results
    results = {'Model_Name': model_name,
               'Train - R2': train_r2,
               'Test - R2': test_r2,
               'Train - Adj-R2': train_adj_r2,
               'Test - Adj-R2': test_adj_r2,
               #'Train - RSS': train_rss,
               #'Test - RSS': test_rss,
               #'Train - MSE': train_mse,
               #'Test - MSE': test_mse,
               'Train - RMSE': train_rmse,
               'Test - RMSE': test_rmse,
               'Train - MAE': train_mae,
               'Test - MAE': test_mae,
               'Train - MAPE': train_mape,
               'Test - MAPE': test_mape}
    
    return pd.DataFrame([results])

In [ ]:
def plot_reg_model_metrics(model_name, y_train, y_test, y_pred_train, y_pred_test):
    
    fig, axs = plt.subplots(2, 3, figsize=(10, 4))
    plt.suptitle(model_name + " Model", fontsize=16)
    
    plt.subplot(2,3,1)
    sns.distplot(y_train)
    sns.distplot(y_pred_train)
    plt.title('Prediction vs Actual (Training)', fontsize=8)
    plt.xlabel(target, fontsize=8)
    plt.ylabel('Density', fontsize=8)
    
    plt.subplot(2,3,2)
    sns.distplot((y_train - y_pred_train))
    plt.title('Error Terms (Training)', fontsize=8)
    plt.xlabel('Errors', fontsize=8)
    plt.ylabel('Density', fontsize=8)
    
    plt.subplot(2,3,3)
    plt.scatter(y_train, y_pred_train)
    plt.plot([y_train.min(),y_train.max()],[y_train.min(),y_train.max()], 'r--')
    plt.title('Train vs Prediction', fontsize=8)
    plt.xlabel('Actual', fontsize=8)
    plt.ylabel('Prediction', fontsize=8)
    
    plt.subplot(2,3,4)
    sns.distplot(y_test)
    sns.distplot(y_pred_test)
    plt.title('Prediction vs Actual (Test)', fontsize=8)
    plt.xlabel(target, fontsize=8)
    plt.ylabel('Density', fontsize=8)
    
    plt.subplot(2,3,5)
    sns.distplot((y_test - y_pred_test))
    plt.title('Error Terms (Test)', fontsize=8)
    plt.xlabel('Errors', fontsize=8)
    plt.ylabel('Density', fontsize=8)
    
    plt.subplot(2,3,6)
    plt.scatter(y_test, y_pred_test)
    plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], 'r--')
    plt.title('Test vs Prediction', fontsize=8)
    plt.xlabel('Actual', fontsize=8)
    plt.ylabel('Prediction', fontsize=8)
    
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_reg_preds(X, y_actual, y_pred, cols=[], first_count=None, rand_count=None):

    if rand_count:
        cols = random.sample(list(X.columns.values), rand_count)
    
    if first_count:
        cols = X.columns.values[:first_count]
    
    num_rows = math.ceil(len(cols)/3)
    
    fig, ax = plt.figure(figsize=[15,3*num_rows])
    for i, col in enumerate(cols):
        plt.subplot(num_rows, 3, i+1)
        plt.suptitle('Prediction vs Actual across Input Features', fontsize=16)

        if X[col].dtype == 'float64':
            sns.scatterplot(y=y_actual, x=X[col], label='Actual', alpha=1)
            sns.scatterplot(y=y_pred, x=X[col], label='Prediction', alpha=0.5)

        elif X[col].dtype == 'bool':
            temp_actual = pd.DataFrame({'Label': 'Actual', 'Value': y_train, col: X_train[col]})
            temp_pred = pd.DataFrame({'Label': 'Prediction', 'Value': y_pred, col: X_train[col]})
            temp = pd.concat([temp_actual, temp_pred], ignore_index=True)
            sns.boxplot(data=temp, x='Label', y='Value', hue=col)
            plt.xlabel(col)
            plt.ylabel(target)

        plt.title(col)
        plt.legend()

    plt.tight_layout()
    plt.show()

    return fig

## f) Model Saving

# 4) XAI - Model Explainability

## a) SHAP

In [22]:
# output various SHAP plots
def run_shap(clf, X, i=0):
    
    #shap.initjs()
    
    explainer = shap.Explainer(clf, X)
    shap_values = explainer(X)
    
    # force plot
    #shap.force_plot(explainer.expected_value, shap_values[i], X[i], feature_names = features)
    
    # waterfall plot
    plt.figure(figsize=(50, 20))
    shap.plots.waterfall(shap_values[i], max_display=25, show=False)
    plt.show()      
    
    # summary plot
    plt.figure(figsize=(50, 20))
    shap.summary_plot(shap_values, X, max_display=25, show=False)
    plt.show()
    
    
    # mean SHAP
    plt.figure(figsize=(50, 20))
    shap.plots.bar(shap_values, max_display=25, show=False)
    plt.show()
    
    # dependence plot
    #shap.dependence_plot(5, shap_values, X, feature_names=['annual_inc','fico'])
    #shap.dependence_plot('annual_inc', shap_values[1], X, interaction_index="annual_inc")
    
    return shap_values

## b) LIME

In [23]:
# output LIME plot
def run_lime(X_train, X_test, clf, i=0):
        
    # get the class names
    class_names = ['No default', 'Default']

    # get the (categorical) feature names
    feature_names = list(X_train.columns)
    
    categorical_names = [col for col in X_train 
                if np.isin(X_train[col].dropna().unique(), [0, 1]).all()]
    categorical_features = [X_train.columns.get_loc(c) for c in categorical_names 
                            if c in X_train]

    # fit the Explainer on the train data
    explainer = LimeTabularExplainer(X_train.values, 
                                     feature_names=feature_names, 
                                     class_names=class_names,
                                     categorical_features=categorical_features, 
                                     categorical_names=categorical_names,
                                     mode='classification')
    
    # local explanation on the i-th instance in test data
    explaination = explainer.explain_instance(X_test.iloc[i], clf.predict_proba)
    explaination.show_in_notebook(show_table = True, show_all = False)

## c) PFI - Permutation Feature Importance

In [24]:
# output PFI plot
def run_pfi(X, y, clf):
    
    perm = PermutationImportance(clf, random_state=42).fit(X, y)
    return eli5.show_weights(perm, feature_names=X.columns.tolist())

# 5) Master-Function (Full Pipeline)

## a) Fit-Predict-Evaluate (single model)

In [25]:
# fit classifier, get preds and probs, evaluate model
def fit_pred_eval(X_test, y_test, X_train, y_train, clf, clf_name, thres=0.5, show=True):
    
    #start_time = time.time()
    
    # fit classifier
    clf_fit = fit_model(X_train, y_train, clf)
    
    # evaluate classifier
    model_eval, cm, roc = eval_model(X_test, y_test, X_train, y_train, clf_fit, clf_name, thres=thres, show=False)
    
    # save model
    pickle_model(clf_fit, clf_name+'_'+datetime.now(pytz.timezone('US/Pacific')).strftime("%d-%m-%Y_%H-%M"))
    
    
    return model_eval.transpose()

## b) End-to-End Pipeline (single/multiple models)

In [26]:
# fun full ML pipeline on all combinations of selected algorithm types, years, sampling and scaling methods, output evaluation summary table
def master_func_eval(clf_list, clf_name_list, sampling_methods, scaling_methods, 
                     target='default', thres=0.5, size=-1, show=False):
    
    # initialize summary tables for model evaluation
    model_eval = pd.DataFrame(index=['RunTime', 'Accuracy', 'Bal_Acc', 'Precision', 'Recall', 'F1', 'AUC'])
    
        
    start_time = time.time()
    
    # data loading and splitting
    X_train_main, X_test_main, y_train_main, y_test_main = load_and_split_data('Data/data_clean_full.csv', target, size=size, year=[yr])

    load_time = time.time() - start_time
    
    # set classifier index
    i = 0

    # loop through all committed classifiers, sampling and scaling methods
    for clf in clf_list:   
        for sampler in sampling_methods:   
            for scaler in scaling_methods:

                start_time = time.time()
                
                # select datasets
                X_train, y_train, X_test, y_test = select_X_y(X_train_main, y_train_main, X_test_main, y_test_main,
                                                              sampler=sampler, scaler=scaler, size=size,
                                                              sampling_strategy=1, version=0, n_neighbors=0)

                # generate classifier name
                clf_name = clf_name_list[i]
                if sampler:
                    clf_name += '_' + sampler

                if scaler:
                    clf_name += '_' + scaler

                # fit, predict and evaluate model
                model_eval[clf_name] = fit_pred_eval(X_train, y_train, X_test, y_test, clf, clf_name, thres=thres, show=show).iloc[:,1]

                # store run time
                model_eval.loc['RunTime', clf_name] = str(timedelta(seconds=time.time()-start_time+load_time))

        # increase classifier index
        i += 1

    # transpose output table
    model_eval = model_eval.transpose()
    
    # split clf_name
    s = model_eval.index.str.split('_')
    
    # initialize new columns
    model_eval[['Clf', 'Scaling', 'Sampling']] = ''
    
    # extract classifier, samping and scaling method
    for i in range(model_eval.shape[0]):
        model_eval['Clf'][i] = s[i][0]
        if len(s[i])==4:
            model_eval['Sampling'][i] = s[i][1]
            model_eval['Scaling'][i] = s[i][2] 
        elif len(s[i])==3:
            if s[i][2] in sampling_methods:
                model_eval['Sampling'][i] = s[i][1]
                model_eval['Scaling'][i] = 'non'
            elif s[i][2] in scaling_methods:
                model_eval['Scaling'][i] = s[i][1]
                model_eval['Sampling'][i] = 'non'  
        else:
            model_eval['Sampling'][i] = 'non'
            model_eval['Scaling'][i] = 'non'
            
    # save output table as csv
    model_eval.to_csv('Master_output/master_output_'+datetime.now(pytz.timezone('US/Pacific')).strftime("%d-%m-%Y_%H-%M")+'.csv')
        
    return model_eval

## c) Output Plots

In [27]:
# plot model performance for all years and classifiers 
def plot_all_yrs(data):
    
    # drop RunTime column
    df = data.drop(['RunTime'], inplace=False, axis=1)
    
    df = pd.melt(df, id_vars =['Clf','Sampling','Scaling'],
                 value_vars =['Accuracy','Bal_Acc','Precision','Recall','F1','AUC'],
                 var_name ='Metric', value_name ='Score')
    
    df_grouped = pd.DataFrame(df.groupby(['Metric'])['Score'].mean())
    df_grouped.reset_index(inplace=True)
    
    # plot mean scores
    plt.figure(figsize=(20, 5))
    plt.subplot(1, 2, 1)
    sns.lineplot(x = 'Year', y = 'Score', hue='Metric', data=df_grouped)
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.subplot(1, 2, 2)
    sns.barplot(x = 'Metric', y = 'Score', hue='Year', data=df_grouped)
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.suptitle('Mean Scores by Year')
    plt.show()
    
    df_grouped = pd.DataFrame(df.groupby(['Year','Clf','Metric'])['Score'].mean())
    df_grouped.reset_index(inplace=True)

    plt.figure(figsize=(20, 15))
    sns.catplot(x='Clf', y='Score',
                col='Metric', kind='bar', col_wrap=6,
                palette='deep', data=df_grouped[df_grouped['Year']==yr])
    plt.suptitle('Year - %s' % (yr), y=1.05, size=24)
    plt.show()